## Aim: use scVI to integrate Achilles tendon data

In [ ]:
# Import dependencies
%matplotlib inline
import os
import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import seaborn as sns
import anndata
import matplotlib.pyplot as plt
import yaml

# Print date and time:
import datetime
e = datetime.datetime.now()
print ("Current date and time = %s" % e)

# Set other settings
sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.set_figure_params(dpi=150, fontsize=10, dpi_save=600)

In [ ]:
# set a working directory
wdir = '/project/tendonhca/ccohen/chromium/analysis/20240221_achilles_python'
os.chdir( wdir )

# create an output directory with today's date and time
year = e.strftime("%Y")
month = e.strftime("%m")
day = e.strftime("%d")
hour = e.strftime('%H')
minute = e.strftime('%M')
dmyt = f'{year}{month}{day}_{hour}-{minute}'
directory = f'{dmyt}_concat_norm.dir'

# folder structures
RESULTS_FOLDERNAME = f'{directory}/results/'
FIGURES_FOLDERNAME = f'{directory}/figures/'

if not os.path.exists(RESULTS_FOLDERNAME):
    os.makedirs(RESULTS_FOLDERNAME)
if not os.path.exists(FIGURES_FOLDERNAME):
    os.makedirs(FIGURES_FOLDERNAME)
    
# Set folder for saving figures into
sc.settings.figdir = FIGURES_FOLDERNAME

directory

In [ ]:
# read in the yaml file 
ini = yaml.safe_load(open('concat_norm.yaml'))
print(yaml.safe_dump(ini))

# list items in the data directory
os.listdir(ini['datadir'])

Read in the objects as a dictionary

In [ ]:
adata_dict = {}
for file in os.listdir(ini['datadir']):
    name = file.replace('.h5ad', '')
    print(name)
    file_path = os.path.join(ini['datadir'], file)
    # print(file_path)
    adata = sc.read_h5ad(file_path)
    adata_dict[name] = adata
adata_dict

In [ ]:
# plot the individual umaps (sanity check that conversion has been performed correctly)
if not os.path.exists(os.path.join(FIGURES_FOLDERNAME, "umap_individual")):
    os.makedirs(os.path.join(FIGURES_FOLDERNAME, "umap_individual"))
for key, adata in adata_dict.items():
    print(key)
    #savename = f'{key}.png'
    sc.pl.umap(adata, color = 'seurat_clusters', save = f'_individual/umap_{key}.png', title = key)

In [ ]:
# plot the individual pcas
if not os.path.exists(os.path.join(FIGURES_FOLDERNAME, "pca_individual")):
    os.makedirs(os.path.join(FIGURES_FOLDERNAME, "pca_individual"))
for key, adata in adata_dict.items():
    print(key)
    sc.pl.pca(adata, color = 'seurat_clusters', save = f'_individual/pca_{key}.png', title = key)

In [ ]:
# check the matrices of one object in adata_dit
print(adata_dict['MSK0785-Ach-MB'].X.max(axis = None))
print(adata_dict['MSK0785-Ach-MB'].layers['counts'].max(axis = None))
print(adata_dict['MSK0785-Ach-MB'].layers['soupX'].max(axis = None))
# X is currently counts

In [ ]:
# concatenate the adata_dict

def concat_filtered_adatafiles(filtered_adata_dict):
    # Extract the values (anndata objects) from the dictionary
    adata_list = list(filtered_adata_dict.values())
    
    # Concatenate the anndata objects
    adata_concat = anndata.concat(
        adata_list,
        join='outer',
        index_unique=None      # Optional: specify a custom index unique function
    )
    
    return adata_concat

In [ ]:
adata_concat = concat_filtered_adatafiles(adata_dict)
adata_concat

In [ ]:
# make names unique
adata_concat.obs_names_make_unique()
adata_concat.obs

In [ ]:
#check the layers
print(adata_concat.X[0:50,0:50].todense())
print(adata_concat.layers['counts'][0:50,0:50].todense())
print(adata_concat.layers['logcounts'][0:50,0:50].todense())

In [ ]:
# remove existing pca and umap from the concatenated object
del adata_concat.obsm['pca'], adata_concat.obsm['umap']

In [ ]:
# remove the previously calculated lognorm layer
del adata_concat.layers['logcounts']

In [ ]:
# add a column in the metadata for patient + sequencing date
adata_concat.obs['patient.seqbatch'] = adata_concat.obs[['patient', 'sequencing_date']].agg('_'.join, axis = 1)

In [ ]:
adata_concat.obs.columns

In [ ]:
# remove surplus obs columns
adata_concat.obs.drop(columns=['subsets_mito_sum',
       'subsets_mito_detected', 'decontX_clusters', 'scDblFinder.cluster',
       'scDblFinder.score', 'scDblFinder.weighted',
       'scDblFinder.difficulty', 'scDblFinder.cxds_score',
       'scDblFinder.mostLikelyOrigin', 'scDblFinder.originAmbiguous',
       'patient.seqbatch', 'n_genes'], inplace=True)

In [ ]:
# if required, set X to be the soupX matrix
# for downstream scVI, then leave X as counts
# adata_concat.X = adata_concat.layers['soupX'].copy()

In [ ]:
# filter out lowly expressed genes (from Alina)
sc.pp.filter_genes(adata_concat, min_counts=30, inplace=True)
sc.pp.filter_cells(adata_concat, min_genes=200)

In [ ]:
adata_concat

In [ ]:
# calculate hvg
sc.pp.highly_variable_genes(adata_concat, 
                            n_top_genes= ini['variable_genes']['n_genes'], 
                            flavor=ini['variable_genes']['flavor'], 
                            batch_key=ini['variable_genes']['batch'], 
                            subset = False)

In [ ]:
# plot hvg
sc.pl.highly_variable_genes(adata_concat)

In [ ]:
# check how many batches each gene was variable in

n_batches = adata_concat.var["highly_variable_nbatches"].value_counts()
ax = n_batches.plot(kind="bar")
n_batches

In [ ]:
# plot_dispersions_by_parameter
def plot_dispersions_by_parameter(adata, countslayers, splitparameter, log_scale=False):
    num_rows = 1
    num_cols = 2

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 5))
    
    for idx, layer in enumerate(countslayers):
        for batch in adata.obs[splitparameter].cat.categories:  # these are the library batches
            adata_batch = adata[adata.obs[splitparameter] == batch, :] # this selects the cells in one batch
            mean_counts = np.mean(adata_batch.layers[layer].toarray(), axis=0) 
                # convert the count matrix to an array and calc the mean of each column (for each gene)
            variance = np.var(adata_batch.layers[layer].toarray(), axis=0) 
                # calc the variance of each column the count matrix (for each gene)
            coefficient_of_variation = (np.sqrt(variance) / mean_counts) * 100

            ax = axes[idx]
            ax.scatter(mean_counts, coefficient_of_variation, # make scatter plot
                       label=f'Batch {batch}', edgecolors='none')

            ax.set_xlabel('Mean Counts')
            ax.set_ylabel('Coefficient of Variation (%)')
            if log_scale:
                ax.set_xscale('log')
                ax.set_yscale('log')
                ax.set_xlabel('Log Mean Counts')
                ax.set_ylabel('Log CV')
            ax.set_title(f'Mean-CV Plot ({layer})')
            ax.legend()

        plt.tight_layout()
        plt.show()

In [ ]:
plot_dispersions_by_parameter(adata_concat, ['counts'], 'sample', log_scale=True)

In [ ]:
# normalise using delta method
sc.pp.normalize_total(adata_concat, target_sum=None, inplace=True)
sc.pp.log1p(adata_concat)
print(adata_concat.X[0:10, 0:10])

In [ ]:
# use the log1PF layer as X
adata_concat.layers["log1p_norm"] = adata_concat.X.copy()

In [ ]:
# scale the data
sc.pp.scale(adata_concat) # scale the data in X
print(adata_concat.X[0:5,0:5])

In [ ]:
# save the scaled data as a new layer
adata_concat.layers['scaled'] = adata_concat.X.copy()

In [ ]:
# calculate PCA
sc.pp.pca(adata_concat, n_comps=40, svd_solver="arpack")

In [ ]:
#plot PCA loadings
sc.pl.pca_loadings(adata_concat, components='1,2,3,4,5,6,7,8')

In [ ]:
# plot the PCA
sc.pl.pca(adata_concat, color=['sample', 'sex', 'age', 'sum', 'microanatomical_site', 'patient', 'sequencing_date'],
          ncols=2, wspace=0.7,
          frameon=False)

In [ ]:
# elbow plot
sc.pl.pca_variance_ratio(adata_concat, n_pcs=40, log=True)

In [ ]:
# calculate neighbours and umap
sc.pp.neighbors(adata_concat, use_rep='X_pca', n_neighbors=15, n_pcs=ini['neighbours']['n_pcs'])
# n_neighbours is 15 by default, this will affect the granularity of the nn graph
# using 20 PCs to match Harmony
sc.tl.umap(adata_concat)

In [ ]:
sc.pl.umap(adata_concat, color=['sex', 'age', 'sum', 'microanatomical_site', 'patient', 'sequencing_date'],
           ncols=2, wspace=0.5, save='_unintegrated.png', frameon=False)

In [ ]:
sc.pl.umap(adata_concat, color='sample',
           ncols=2, wspace=0.5, save='_unintegrated_sample.svg', frameon=False)

In [ ]:
adata_concat

In [ ]:
# save the concatenated object
path = os.path.join(RESULTS_FOLDERNAME, 'merged_normalised.h5ad')
adata_concat.write(path)


In [ ]:
# for testing, subset the object to only 2 samples
adata_subset = adata_concat[(adata_concat.obs['sample'] == 'MSK0785-Ach-MB') |
              (adata_concat.obs['sample'] == 'MSK1250-Ach-MB2') ]
adata_subset

In [ ]:
# save the object
adata_subset.write(os.path.join(RESULTS_FOLDERNAME, 'Achilles_subset.h5ad'))